In [1]:
# http://pytorch.org/
try:
    import torch
except ImportError:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
    
    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl
    # !pip install torchvision
    # subprocess.check_call(['pip', 'install', '-q', f'http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl'])
    import torch

In [2]:
try:
    import torchtext
except ImportError:
    # subprocess.check_call(['pip', 'install', 'torchtext'])
    !pip install torchtext
try:
    import spacy
except ImportError:
    # subprocess.check_call(['pip', 'install', 'spacy'])
    !pip install spacy

In [3]:
import torch 
from torchtext import data
torch.__version__

'0.4.1'

In [4]:
import spacy.cli
spacy.cli.download('en')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [5]:
SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)

In [6]:
from torchtext import datasets
train, test = datasets.IMDB.splits(TEXT, LABEL)

In [7]:
import random
train, valid = train.split(split_ratio = 0.8, random_state=random.seed(SEED))

In [8]:
TEXT.build_vocab(train, max_size=25000, vectors="glove.6B.100d")
LABEL.build_vocab(train)

In [9]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size=BATCH_SIZE,
    device=device)

In [10]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(x))
        
        #embedded = [sent len, batch size, emb dim]
        
        output, (hidden, cell) = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim * num directions]
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden.squeeze(0))

In [11]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

lstmmodel = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)

In [12]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [13]:
lstmmodel.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.3018,  0.6112,  0.7165,  ..., -0.2016,  0.0172,  0.4952],
        [ 0.5744, -0.7163, -0.1376,  ..., -0.0314,  0.9674, -0.5805],
        [ 0.6607,  0.0144,  0.2799,  ...,  1.1042, -0.5144,  0.2072]])

In [14]:
import torch.optim as optim

optimizer = optim.Adam(lstmmodel.parameters())
# optimizer = optim.SGD
criterion = nn.BCEWithLogitsLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

lstmmodel = lstmmodel.to(device)
criterion = criterion.to(device)

In [15]:
import torch.nn.functional as F

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(F.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [16]:
def train_model(model, train_iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in train_iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(train_iterator), epoch_acc / len(train_iterator)

In [17]:
def evaluate_model(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [18]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train_model(lstmmodel, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate_model(lstmmodel, valid_iterator, criterion)
    
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc*100:.2f}%, Val. Loss: {valid_loss:.3f}, Val. Acc: {valid_acc*100:.2f}%')

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 01, Train Loss: 0.665, Train Acc: 59.30%, Val. Loss: 0.679, Val. Acc: 67.07%
Epoch: 02, Train Loss: 0.619, Train Acc: 65.15%, Val. Loss: 0.670, Val. Acc: 57.95%
Epoch: 03, Train Loss: 0.530, Train Acc: 74.57%, Val. Loss: 0.349, Val. Acc: 86.00%
Epoch: 04, Train Loss: 0.379, Train Acc: 84.77%, Val. Loss: 0.394, Val. Acc: 84.12%
Epoch: 05, Train Loss: 0.285, Train Acc: 89.08%, Val. Loss: 0.312, Val. Acc: 88.43%


In [19]:
test_loss_rnn, test_acc_rnn = evaluate_model(lstmmodel, test_iterator, criterion)

print(f'| Test Loss: {test_loss_rnn:.3f} | Test Acc: {test_acc_rnn*100:.2f}% |')

| Test Loss: 0.359 | Test Acc: 86.19% |


In [20]:
torch.save(lstmmodel.state_dict(),'model_lstm_new.pth')